In [1]:
%pip install llama-index-llms-openai
!pip install llama-index

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.ERROR)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))


Note: you may need to restart the kernel to use updated packages.


In [2]:
from llama_index.core import SimpleDirectoryReader

# documents = SimpleDirectoryReader(input_files=['../data/paul_graham_essay3.txt']).load_data()
documents = SimpleDirectoryReader(input_files=['../data/2022 Q3 AAPL.pdf']).load_data()

In [3]:
!pip install llama-index-embeddings-ollama

In [4]:
from llama_index.core import Settings
from llama_index.llms.ollama import  Ollama
Settings.llm = Ollama(model='llama3.2:latest', base_url='http://localhost:11434',temperature=0.1)
ollama_llm = Ollama(model='llama3.2:latest', base_url='http://localhost:11434',temperature=0.1)

In [5]:
!pip install llama-index-embeddings-ollama

In [6]:
from llama_index.embeddings.ollama import OllamaEmbedding

In [7]:
ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text:latest",  # Replace with your desired model
    base_url="http://localhost:11434",  # Ensure Ollama is running at this endpoint
    ollama_additional_kwargs={"mirostat": 0} #Mirostat is a technique for controlling perplexity and balancing the text generation process in large language models (LLMs).
)  

In [8]:
len(documents)

28

In [9]:
documents[0].dict().keys()


dict_keys(['id_', 'embedding', 'metadata', 'excluded_embed_metadata_keys', 'excluded_llm_metadata_keys', 'relationships', 'text', 'mimetype', 'start_char_idx', 'end_char_idx', 'text_template', 'metadata_template', 'metadata_seperator', 'class_name'])

In [10]:
from llama_index.core import DocumentSummaryIndex
from llama_index.core import get_response_synthesizer


In [ ]:
# Configure response synthesizer with tree summarization
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
    llm=ollama_llm

)

# Create DocumentSummaryIndex
doc_summary_index = DocumentSummaryIndex.from_documents(
    documents,
    response_synthesizer=response_synthesizer,
    embed_model=ollama_embedding,
    show_progress=True
)

Parsing nodes:   0%|          | 0/28 [00:00<?, ?it/s]

Summarizing documents:   0%|          | 0/28 [00:00<?, ?it/s]

current doc id: 7c9aae22-c3d3-48e2-bcdd-f25dab461062
current doc id: d3c674c6-0af6-4d99-83dd-7f17d4076899
current doc id: b2809366-aa21-4c10-965d-e0c52b9cf746
current doc id: 8c7237a0-9ba6-423d-a4d4-f252ed6fa807
current doc id: 7491894f-0771-4090-8bab-572bed792f78
current doc id: 1fb68af1-9033-4a3f-9744-a5317cd1bfde
current doc id: 0877b316-60cd-4293-851a-e4fa59fa4800
current doc id: c730d5a8-3d0e-4350-ab40-4a43fbeee667
current doc id: 6ed4b814-e74c-41c5-9cf1-ed1ff145fb3a
current doc id: 6520c245-87ba-40e2-acdf-3cbeae8eb6c9
current doc id: f2c1f7e3-6b1a-4f35-a020-7c6e090035e7
current doc id: 15d26556-104e-4f5b-855d-e2f952d3a923
current doc id: 0b4f7bb6-598c-4811-9023-5596054a4527
current doc id: 0c90799b-9f58-4dc2-be44-fae6ce368059
current doc id: c3b129b2-8328-4ba0-961e-b24944e27e5f
current doc id: 8b52ca3e-433d-4ed8-9a10-b6ac0a386e7b
current doc id: 38113446-b0f5-4c0d-abe8-d2f60fd8fe99
current doc id: e480fe72-1070-4efb-b873-e17bf2397181
current doc id: 4459a0c5-00e6-436c-9c8b-e77010

In [ ]:
summary = doc_summary_index.get_document_summary(documents[0].doc_id)


In [ ]:
print("\n Summary : " + summary)


In [ ]:
# Print the number of loaded documents
print(f"Number of documents loaded: {len(documents)}")

# Generate summaries for all loaded documents
for i, document in enumerate(documents):
    print(f"Summary for Document {i + 1}:")
    summary = doc_summary_index.get_document_summary(document.doc_id)
    print(summary)
    print("------------------------------------")

In [ ]:
doc_summary_index


In [ ]:
doc_summary_index.index_struct.to_dict().keys()


In [ ]:
doc_summary_index.index_struct.doc_id_to_summary_id



In [ ]:
doc_summary_index.index_struct.node_id_to_summary_id


In [ ]:
doc_summary_index.index_struct.summary_id_to_node_ids

In [ ]:
doc_summary_index.vector_store.persist("document_summary_index_vector_store.json")


In [ ]:
doc_summary_index.docstore.persist("document_summary_index_doc_store.json")


In [ ]:
doc_summary_llm_retriever = doc_summary_index.as_retriever(retriever_mode='llm')


In [ ]:
llm_mode_nodes = doc_summary_llm_retriever.retrieve("Provide the profit and sales number for this result period")


In [ ]:
len(llm_mode_nodes)

In [ ]:
for node in nodes:
  print("Node Id:",node.id_)
  print("Metadata:",node.metadata)
  print("------------------------------------")
  print("Metadata:",node.text)
  print("------------------------------------")

In [ ]:
doc_summary_embed_retriever = doc_summary_index.as_retriever(retriever_mode='embedding', similarity_top_k=3)


In [ ]:
embedding_nodes = doc_summary_embed_retriever.retrieve("Provide the profit and sales number for this result period")


In [ ]:
len(embedding_nodes)

In [ ]:
for node in embedding_nodes:
  print("Node Id:",node.id_)
  print("Metadata:",node.metadata)
  print("------------------------------------")
  print("Metadata:",node.text)
  print("------------------------------------")